In [1]:
import pandas as pd
import numpy as np
import os
import trimesh
from meshparty import trimesh_io, trimesh_vtk, skeleton, utils

In [2]:
# Load astrocyte cellids into a list
astrocytes = pd.read_csv('data/astrocytes.csv', index_col=[0])
astro_list = astrocytes.cell_segid.to_list()
len(astro_list)

44

In [3]:
astrocytes

,centroid_x,centroid_y,centroid_z,cell_segid,cell_type,cell_subtype
0,88240,60656,256,648518346349517321,glia,astrocyte
1,72368,56912,590,648518346341392909,glia,astrocyte
4,72704,54688,506,648518346349536799,glia,astrocyte
5,74576,75520,1163,648518346349525537,glia,astrocyte
6,79328,67360,445,648518346349525544,glia,astrocyte
10,95184,37376,1808,648518346349526583,glia,astrocyte
12,83952,46640,1633,648518346349490239,glia,astrocyte
18,80272,56768,2068,648518346349524070,glia,astrocyte
20,101120,49760,60,648518346349386860,glia,astrocyte
23,73712,73056,1123,648518346349508722,glia,astrocyte


In [4]:
# create vasculature list

celltypes = pd.read_csv("data/220309_cell_type_classification.csv", index_col=[0])
vasc = celltypes[celltypes.cell_type == 'vasculature']
vasc_list = vasc.cell_segid.to_list()
len(vasc_list)

72

In [5]:
mesh_dir = 'data/neuron_meshes_v185/dec/' # or change to your desired folder
seg_source = "precomputed://https://storage.googleapis.com/microns_public_datasets/pinky100_v185/seg"
mm = trimesh_io.MeshMeta(cv_path=seg_source,
                         disk_cache_path=mesh_dir, 
                         cache_size=20)

In [6]:
# generate trimesh objects of vascular and astro cells and place in a dictionary

def update_mesh_dictionary(cell_list, mesh_dictionary, prefix, mesh_dir):
    for cellid in cell_list:
        mesh_file = os.path.join(mesh_dir, str(cellid) + '.ply')
        mesh_dictionary[prefix + str(cellid)] = trimesh.load_mesh(mesh_file)

# Initialize the mesh dictionary
mesh_dictionary = {}

# Update the dictionary with astro_list
update_mesh_dictionary(astro_list, mesh_dictionary, 'astro_', mesh_dir)

# Update the dictionary with vasc_list
update_mesh_dictionary(vasc_list, mesh_dictionary, 'vasc_', mesh_dir)


In [7]:
len(mesh_dictionary)

61

In [8]:
# Render all astros with the same settings and pull the viewpoint centroid from an index in astro_list (default index [0])

# Define colors and opacities
astro_color = [0.5, 0.5, 0.5]  # Neutral gray
astro_opac = 0.005

vasc_color = [0.5, 0.0, 0.0]  # Dark red
vasc_opac = 0.1

# Creating mesh actors with specified colors and opacities without modifying the mesh_dictionary
mesh_actor = {}
for cellid in astro_list:
    cell_key = 'astro_' + str(cellid)
    if cell_key in mesh_dictionary:
        mesh_actor[cell_key] = trimesh_vtk.mesh_actor(
            mesh_dictionary[cell_key], 
            opacity=astro_opac, 
            color=astro_color
        )

for cellid in vasc_list:
    cell_key = 'vasc_' + str(cellid)
    if cell_key in mesh_dictionary:
        mesh_actor[cell_key] = trimesh_vtk.mesh_actor(
            mesh_dictionary[cell_key], 
            opacity=vasc_opac, 
            color=vasc_color
        )

locals().update(mesh_actor)

# Creating a camera object and defining the view
if 'astro_' + str(astro_list[0]) in mesh_dictionary:
    camera = trimesh_vtk.oriented_camera(mesh_dictionary['astro_' + str(astro_list[0])].centroid, backoff=400)
else:
    print(f"First cell key astro_{astro_list[0]} not found in mesh dictionary.")

# Render the actors, will open a pop-up Python window
trimesh_vtk.render_actors(mesh_actor.values(), camera=camera)


setting up renderer
done setting up
actors added
camera set
render done
finalizing..


<vtkmodules.vtkRenderingOpenGL2.vtkOpenGLRenderer(0x000001A013C2BF80) at 0x000001A017FC8A08>

In [9]:
# Render all astros with the same settings and calculate a global mean of the viewpoint centroid

# Define colors and opacities
astro_color = [0.5, 0.5, 0.5]  # Neutral gray
astro_opac = 0.005

vasc_color = [0.5, 0.0, 0.0]  # Dark red
vasc_opac = 0.1

# Creating mesh actors with specified colors and opacities without modifying the mesh_dictionary
mesh_actor = {}
centroids = []

for cellid in astro_list:
    cell_key = 'astro_' + str(cellid)
    if cell_key in mesh_dictionary:
        mesh_actor[cell_key] = trimesh_vtk.mesh_actor(
            mesh_dictionary[cell_key], 
            opacity=astro_opac, 
            color=astro_color
        )
        centroids.append(mesh_dictionary[cell_key].centroid)

for cellid in vasc_list:
    cell_key = 'vasc_' + str(cellid)
    if cell_key in mesh_dictionary:
        mesh_actor[cell_key] = trimesh_vtk.mesh_actor(
            mesh_dictionary[cell_key], 
            opacity=vasc_opac, 
            color=vasc_color
        )
        centroids.append(mesh_dictionary[cell_key].centroid)

locals().update(mesh_actor)

# Calculate the overall centroid
if centroids:
    overall_centroid = np.mean(centroids, axis=0)
else:
    overall_centroid = np.array([0, 0, 0])  # Fallback if no centroids are found

# Creating a camera object and defining the view
camera = trimesh_vtk.oriented_camera(overall_centroid, backoff=400)

# Render the actors, will open a pop-up Python window
trimesh_vtk.render_actors(mesh_actor.values(), camera=camera)


setting up renderer
done setting up
actors added
camera set
render done
finalizing..


<vtkmodules.vtkRenderingOpenGL2.vtkOpenGLRenderer(0x000001A013C2C5D0) at 0x000001A017FC30A8>

In [10]:
# Render vasculature and all astros, using random astro colors and a range of astro opacities

# Define colors and opacities for vasc_list
vasc_color = [0.5, 0.0, 0.0]  # Dark red
vasc_opac = 0.1

# Creating mesh actors with specified colors and opacities without modifying the mesh_dictionary
mesh_actor = {}
centroids = []

for cellid in astro_list:
    cell_key = 'astro_' + str(cellid)
    if cell_key in mesh_dictionary:
        # Randomize astro_color and astro_opac for each mesh
        astro_color = np.random.random(size=3)  # Random color
        astro_opac = np.random.uniform(0.01, 0.075)  # Random opacity between 0.005 and 0.05
        mesh_actor[cell_key] = trimesh_vtk.mesh_actor(
            mesh_dictionary[cell_key], 
            opacity=astro_opac, 
            color=astro_color
        )
        centroids.append(mesh_dictionary[cell_key].centroid)

for cellid in vasc_list:
    cell_key = 'vasc_' + str(cellid)
    if cell_key in mesh_dictionary:
        mesh_actor[cell_key] = trimesh_vtk.mesh_actor(
            mesh_dictionary[cell_key], 
            opacity=vasc_opac, 
            color=vasc_color
        )
        centroids.append(mesh_dictionary[cell_key].centroid)

locals().update(mesh_actor)

# Calculate the overall centroid
if centroids:
    overall_centroid = np.mean(centroids, axis=0)
else:
    overall_centroid = np.array([0, 0, 0])  # Fallback if no centroids are found

# Creating a camera object and defining the view
camera = trimesh_vtk.oriented_camera(overall_centroid, backoff=400)

# Render the actors, will open a pop-up Python window
trimesh_vtk.render_actors(mesh_actor.values(), camera=camera)


setting up renderer
done setting up
actors added
camera set
render done
finalizing..


<vtkmodules.vtkRenderingOpenGL2.vtkOpenGLRenderer(0x000001A013C2F200) at 0x000001A0043535E8>

In [11]:
# Render one astro and all the vasculature 

# Define colors and opacities for vasc_list
vasc_color = [0.5, 0.0, 0.0]  # Dark red
vasc_opac = 0.1

# Define the specific astro cell ID and its properties
specific_astro_id = 648518346349527319
astro_color = np.random.random(size=3)  # Random color
astro_opac = 0.25

# Creating mesh actors with specified colors and opacities without modifying the mesh_dictionary
mesh_actor = {}
centroids = []

# Add the specific astro mesh
astro_key = 'astro_' + str(specific_astro_id)
if astro_key in mesh_dictionary:
    mesh_actor[astro_key] = trimesh_vtk.mesh_actor(
        mesh_dictionary[astro_key], 
        opacity=astro_opac, 
        color=astro_color
    )
    # Use the centroid of this specific astro mesh for the camera
    overall_centroid = mesh_dictionary[astro_key].centroid

# Add all the vasc meshes
for cellid in vasc_list:
    cell_key = 'vasc_' + str(cellid)
    if cell_key in mesh_dictionary:
        mesh_actor[cell_key] = trimesh_vtk.mesh_actor(
            mesh_dictionary[cell_key], 
            opacity=vasc_opac, 
            color=vasc_color
        )
        centroids.append(mesh_dictionary[cell_key].centroid)

locals().update(mesh_actor)

# Creating a camera object and defining the view
camera = trimesh_vtk.oriented_camera(overall_centroid, backoff=150)

# Render the actors, will open a pop-up Python window
trimesh_vtk.render_actors(mesh_actor.values(), camera=camera)


setting up renderer
done setting up
actors added
camera set
render done
finalizing..


<vtkmodules.vtkRenderingOpenGL2.vtkOpenGLRenderer(0x000001A013C2D270) at 0x000001A004353588>